### nn.module
- 按自己理解去解读

- 李沐大佬自定义的nn.module继承了self.training变量
- 下面是看到怎么定义的

In [3]:
from torch.nn import Module

```python
 def __init__(self) -> None:
        """
        Initializes internal Module state, shared by both nn.Module and ScriptModule.
        """
        torch._C._log_api_usage_once("python.nn_module")

        self.training = True
        self._parameters: Dict[str, Optional[Parameter]] = OrderedDict()
        self._buffers: Dict[str, Optional[Tensor]] = OrderedDict()
        self._non_persistent_buffers_set: Set[str] = set()
        self._backward_hooks: Dict[int, Callable] = OrderedDict()
        self._is_full_backward_hook = None
        self._forward_hooks: Dict[int, Callable] = OrderedDict()
        self._forward_pre_hooks: Dict[int, Callable] = OrderedDict()
        self._state_dict_hooks: Dict[int, Callable] = OrderedDict()
        self._load_state_dict_pre_hooks: Dict[int, Callable] = OrderedDict()
        self._load_state_dict_post_hooks: Dict[int, Callable] = OrderedDict()
        self._modules: Dict[str, Optional['Module']] = OrderedDict()
```

- 可以看到self.training = True, 只是个布尔值, 
- 结合李沐大佬的代码,我们可以判断training应该是控制网络是处于训练状态还是预测状态的
- (其实李沐大佬应该不止一次使用的次功能,但是直到现在才发现,惭愧)

结合我们之前训练完毕的时候,一般是使用net.eval()函数来结束网络的,那net.eval又是如何实现这一功能呢?

Module.eval()
```python
def eval(self: T) -> T:
    return self.train(False)
```

```python
def train(self: T, mode: bool = True) -> T:
    if not isinstance(mode, bool):
        raise ValueError("training mode is expected to be boolean")
    self.training = mode
    for module in self.children():
        module.train(mode)
    return self
```

- 可以看到net.eval()等价于Module.training = False !
- 可以看出,net.eval本质上是通过控制nn.Module的training变量,来实现对整个网络处于训练还是预测模式进行控制
